In [1]:
import numpy as np
import scipy as sp
import galois
from wiedeman_solver import WiedemanSolver
from tqdm import tqdm
from datetime import datetime

In [2]:
rng = np.random.default_rng(42)

In [3]:
solver = WiedemanSolver(3)

In [4]:
modulus = solver.modulus
field = solver.field

In [5]:
def generate_non_singular_sparse_matrix(n, density=0.5):
    A = sp.sparse.random(n, n, density=density, format='csr', dtype=int, rng=rng)
    A = sp.sparse.csr_matrix(A.toarray() % modulus)
    while np.linalg.det(A.toarray().view(field)) == 0:
        A = sp.sparse.random(n, n, density=density, format='csr', dtype=int, rng=rng)
        A = sp.sparse.csr_matrix(A.toarray() % modulus)
    return A

In [6]:
def generate_sparse_matrix(n: int, m: int, density: float):
    entries_number = n * m
    nnz = int(entries_number * density)
    rows = np.random.choice(n, size=nnz)
    cols = np.random.choice(m, size=nnz)
    data = np.random.randint(1, 3, size=nnz)
    M = sp.sparse.coo_matrix((data, (rows, cols)), shape=(n, m))
    return sp.sparse.csr_matrix(M)

In [11]:
epochs = 10

## Full rank

### 500x500

In [12]:
n = 500
density = 0.3
iterations = []
times = []
for _ in tqdm(range(epochs)):
    A = generate_non_singular_sparse_matrix(n, density=0.5)
    b = field.Random(n)
    time = datetime.now()
    x, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:06<00:00,  1.53it/s]


In [13]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 1, 2, 1, 1, 1, 2, 1]
1.2
0.142


### 1000x1000

In [14]:
n = 1000
density = 0.3
iterations = []
times = []
for _ in tqdm(range(epochs)):
    A = generate_non_singular_sparse_matrix(n, density=0.5)
    b = field.Random(n)
    time = datetime.now()
    x, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:45<00:00,  4.52s/it]


In [15]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 2, 2, 1, 1, 1, 1, 2, 1, 1]
1.3
0.699


## Singular, but RHS in image

## Rank <= 1000

#### Density = 0.5

In [16]:
n = 1000
m = 1000
density = 0.5
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:34<00:00,  3.42s/it]


In [17]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 2, 2, 1, 3, 1, 1, 2, 2]
1.6
3.224


#### Density = 0.3

In [18]:
n = 1000
m = 1000
density = 0.3
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds()) 
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:32<00:00,  3.27s/it]


In [19]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[3, 1, 1, 1, 1, 1, 2, 1, 1, 1]
1.3
3.163


#### Density = 0.1

In [20]:
n = 1000
m = 1000
density = 0.1
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:31<00:00,  3.16s/it]


In [21]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
1.1
3.121


## Rank <= 500

### Density = 0.5

In [22]:
n = 1000
m = 500
density = 0.5
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:46<00:00,  4.68s/it]


In [23]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 3, 1, 1, 5, 1, 1, 2]
1.7
4.578


### Density = 0.3

In [24]:
n = 1000
m = 500
density = 0.3
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:44<00:00,  4.40s/it]


In [25]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 1, 4, 2, 1, 1, 1, 1]
1.4
4.341


### Density = 0.1

In [26]:
n = 1000
m = 500
density = 0.1
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:36<00:00,  3.60s/it]


In [27]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 2, 2, 1, 1, 1, 2, 1, 3]
1.5
3.574


## Rank <= 100

### Density = 0.5

In [28]:
n = 1000
m = 100
density = 0.5
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:43<00:00,  4.38s/it]


In [29]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[2, 1, 1, 1, 1, 3, 1, 1, 1, 1]
1.3
4.349


### Density = 0.3

In [30]:
n = 1000
m = 100
density = 0.3
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:30<00:00,  3.09s/it]


In [31]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 1, 1, 3, 2, 1, 1, 2]
1.4
3.065


### Density = 0.1

In [32]:
n = 1000
m = 100
density = 0.1
iterations = []
times = []
for _ in tqdm(range(epochs)):
    M = generate_sparse_matrix(n, m, density)
    A = M @ M.T
    x = field.Random(A.shape[1])
    b = solver.matvec(A, x)
    time = datetime.now()
    x_, _ = solver.wiedeman_singular(A, b, verbose=False)
    times.append((datetime.now() - time).total_seconds())
    if not np.all(solver.matvec(A, x_) == b):
        print("Fail!")
        break
    iterations.append(solver.k)

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [33]:
print(iterations)
print(sum(iterations) / 10)
print(round(sum(times) / 10, 3))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1.0
1.019
